Request authorization from the user to access Spotify resources on the user's behalf. 
To do this, send a GET request to the /authorize endpoint. 

:return: \
  If success, user is redirected back to the redirect_uri (contains 2 params: code and state). \
  If failure, the response string contains 2 params: error and state.

In [0]:
%pip install python-dotenv

In [0]:
%restart_python

In [0]:

dbutils.secrets.listScopes()


In [0]:
# Define widget (default value = "")
dbutils.widgets.text("auth_code", "", "Enter your authorization code from Spotify")

# Retrieve widget value
auth_code = dbutils.widgets.get("auth_code")
print(f"Authorization code received: {auth_code}")


In [0]:
import os
from dotenv import load_dotenv
import secrets
import urllib
import requests
import base64


In [0]:
load_dotenv()

RAW_FOLDER_PATH = "/mnt/myspotify/raw"
PROCESSED_FOLDER_PATH = "/mnt/myspotify/processed"
PRESENTATION_FOLDER_PATH = "/mnt/myspotify/presentation"

client_id = os.getenv("CLIENT_ID")
client_secret = dbutils.secrets.get(key="client-secret", scope="spotify-scope")
flask_secret_key = os.getenv("FLASK_SECRET_KEY")
auth_url_base = os.getenv("AUTH_URL")
api_url_base = os.getenv("API_URL")
token_url = os.getenv("TOKEN_URL")
redirect_uri = os.getenv("REDIRECT_URI")

In [0]:
from IPython.display import display, HTML

scope = 'user-library-read user-read-recently-played user-top-read playlist-read-private'
state = secrets.token_urlsafe(16)

params = {
    'client_id': client_id,
    'response_type': 'code',
    'redirect_uri': redirect_uri,
    'scope': scope,
    'state': state,
    'show_dialog': 'true'
}

auth_url = f"{auth_url_base}?{urllib.parse.urlencode(params)}"

# print(f"Authorization URL: {auth_url}")

display(HTML(f"<a href='{auth_url}'>Click here to authenticate with Spotify</a>"))

In [0]:

# if success
req_body = {
    "code": auth_code,
    "grant_type": "authorization_code",
    "redirect_uri": redirect_uri,
}

# Encode client_id and client_secret in base64
b64_auth = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()
headers = {
    "Authorization": f"Basic {b64_auth}",
    "Content-Type": "application/x-www-form-urlencoded",
}

# Exchange code for tokens by sending POST request 
response = requests.post(
    token_url,
    data = req_body,
    headers = headers
)

token_info = response.json()

In [0]:
token_info